In [55]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim
from sklearn.cluster import KMeans

from collections import defaultdict
from dataLoad import trainValidLoad
from metric import ndcg_calculator, hit_at_k

path= "../dataset/"
train, train_valid, sample_sumbission = trainValidLoad(path)

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt

okt = Okt()

def tokenize(text):
    return okt.morphs(text)

item_data["item_name_tokenized"] = item_data["item_name"].apply(lambda x: tokenize(x))

In [2]:
item_data = pd.read_parquet("../data/item_names_table.parquet")

In [15]:
all_tokens = []
for tokens in item_data["item_name_tokenized"]:
    all_tokens.extend(tokens)

In [17]:
%%time
w2vec = Word2Vec(sentences=all_tokens, vector_size=32, min_count=1, workers=4)

CPU times: user 3min 4s, sys: 1.62 s, total: 3min 6s
Wall time: 1min 56s


In [23]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(w2vec.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0)
    except IndexError as e:
        pass

In [24]:
item_data["wv"] = item_data["item_name_tokenized"].map(get_sentence_mean_vector)

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [48]:
word_vectors = item_data.wv.to_list() 

flattened_word_vectors = []
for sub_list in word_vectors:
    if isinstance(sub_list, np.ndarray):
        flattened_word_vectors.extend(sub_list.tolist())
    else:
        pass

In [53]:
from sklearn.cluster import KMeans
import time

word_vectors = flattened_word_vectors
num_clusters = 10 

In [56]:
word_vectors = np.array(word_vectors).reshape(-1, 1)

In [57]:
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)
item_data["category"] = idx

ValueError: Length of values (26345920) does not match length of index (970245)

In [ ]:
from sklearn.manifold import TSNE

X = item_data["wv"].to_list()
y = item_data["category"].to_list()

import os.path
import pickle

tsne_filepath = "tsne3000.pkl"

if not os.path.exists(tsne_filepath):
    tsne = TSNE(random_state=42)
    tsne_points = tsne.fit_transform(X)
    with open(tsne_filepath, "wb+") as f:
        pickle.dump(tsne_points, f)
else:  
    with open(tsne_filepath, "rb") as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=range(len(X)), columns=["x_coord", "y_coord"])
tsne_df["item_name_tokenized"] = item_data["user.description"].to_list()
tsne_df["cluster_no"] = y

In [ ]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(model.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0)
    except IndexError as e:
        pass

In [ ]:
item_data["wv"] = item_data["item_name_tokenized"].map(get_sentence_mean_vector)

In [ ]:
import re

def remove_special_characters(text):
    return [word for word in text if re.match("^[가-힣]+$", word) is not None]

item_data["item_name_tokenized"] = item_data["item_name_tokenized"].apply(remove_special_characters)